## Usage and Customization

### How to Use This Notebook

1. **Run All Cells**: Execute cells sequentially from top to bottom
2. **Modify Parameters**: Edit the `DKWController` parameters to experiment with different settings:
   - `epsilon_target`: Change the target error threshold
   - `delta`: Adjust confidence level for DKW bounds
   - `min_samples`: Set minimum observations before decisions
   - `hysteresis`: Control sensitivity to state changes

3. **Custom Data**: Replace `sample_data` with your own dataset (must have `id` and `difficulty` fields)

### Example Modifications

```python
# Example: More conservative controller
controller = DKWController(
    epsilon_target=0.05,  # Lower error tolerance
    min_samples=200,      # More observations needed
    hysteresis=0.1        # More resistance to state changes
)

# Example: Custom dataset
custom_data = [
    {\"id\": \"test_1\", \"difficulty\": 0.3},
    {\"id\": \"test_2\", \"difficulty\": 0.7},
    # ... add more examples
]
```

### Original Script Equivalent

This notebook replaces the original `method.py` script and provides the same functionality without external file dependencies. The results can be saved to JSON format for compatibility with the original workflow.

---
**Note**: This is a self-contained interactive version of experiment_001/method.py with inlined data for easy experimentation and learning."

In [ ]:
# Detailed analysis
print("🔍 DETAILED ANALYSIS")
print("=" * 50)

# Error rates by method
baseline_errors = sum(1 for r in results["baseline"] if r["error"])
proposed_errors = sum(1 for r in results["proposed"] if r["error"])
total_examples = len(results["baseline"])

print(f"\n📊 Error Rates:")
print(f"  Total examples: {total_examples}")
print(f"  Baseline errors: {baseline_errors}/{total_examples} ({baseline_errors/total_examples:.1%})")
print(f"  Proposed errors: {proposed_errors}/{total_examples} ({proposed_errors/total_examples:.1%})")

# Decision distribution
print(f"\n🎯 Decision Distribution:")
print(f"  Baseline: {total_examples - baseline_fusions} fission, {baseline_fusions} fusion")
print(f"  Proposed: {total_examples - proposed_fusions} fission, {proposed_fusions} fusion")

# Save results (equivalent to original script's file output)
print(f"\n💾 Results saved to variable 'results'")
print("   To save to JSON file, use:")
print("   import json")
print("   with open('method_out.json', 'w') as f:")
print("       json.dump(results, f, indent=2)")

# Pretty print the results structure
print(f"\n📋 Results structure preview:")
print("results = {")
print(f"  'baseline': [{len(results['baseline'])} items],")
print(f"  'proposed': [{len(results['proposed'])} items]")
print("}")

# Show first few results as example
print(f"\nFirst few results:")
print(f"  Baseline[0]: {results['baseline'][0]}")
print(f"  Proposed[0]: {results['proposed'][0]}")

## Results Analysis and Export

The following cell provides additional analysis and shows how to export results in the same format as the original script.

In [ ]:
# Run the experiment
results = run_experiment(sample_data)

# Display results
print("🔬 EXPERIMENT RESULTS")
print("=" * 50)

print("\n📊 BASELINE (Always Fission):")
for result in results["baseline"]:
    print(f"  {result['id']}: {result['decision']} | Error: {result['error']}")

print("\n🧠 PROPOSED (DKW Controller):")
for result in results["proposed"]:
    print(f"  {result['id']}: {result['decision']} | Error: {result['error']}")

# Analysis
baseline_fusions = sum(1 for r in results["baseline"] if r["decision"] == "fusion")
proposed_fusions = sum(1 for r in results["proposed"] if r["decision"] == "fusion")

print(f"\n📈 SUMMARY:")
print(f"  Baseline fusion decisions: {baseline_fusions}/{len(results['baseline'])}")
print(f"  Proposed fusion decisions: {proposed_fusions}/{len(results['proposed'])}")
print(f"  DKW controller adaptivity: {'✓ Adaptive' if proposed_fusions > baseline_fusions else '✗ Conservative'}")

## Running the Experiment

Now let's run the experiment with our sample data and examine the results. The experiment will compare:

- **Baseline**: Always chooses "fission" (conservative approach)
- **Proposed**: Uses DKW controller to adaptively choose between fusion and fission

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inline data."""
    controller = DKWController()
    results = {"baseline": [], "proposed": []}
    
    # Set random seed for reproducible results
    np.random.seed(42)

    for example in data:
        # Simulate error occurrence based on difficulty
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
        })

    return results

print("✓ Experiment function defined successfully!")

## Experiment Function

The experiment function simulates running both the DKW controller (proposed method) and a baseline conservative approach. For each example, it:

1. Simulates an error occurrence based on the difficulty level
2. Feeds the error to the DKW controller and gets its decision
3. Records results for both the proposed method and a conservative baseline

In [ ]:
# Sample dataset - inlined to make notebook self-contained
# This replaces reading from "../dataset_001/data_out.json"
sample_data = [
    {"id": "example_000", "difficulty": 0.1},  # Low difficulty -> low error probability
    {"id": "example_001", "difficulty": 0.05}, # Very low difficulty  
    {"id": "example_002", "difficulty": 0.8},  # High difficulty -> high error probability
    {"id": "example_003", "difficulty": 0.2},  # Medium-low difficulty
    {"id": "example_004", "difficulty": 0.15}, # Low difficulty
    {"id": "example_005", "difficulty": 0.6},  # Medium-high difficulty
    {"id": "example_006", "difficulty": 0.3},  # Medium difficulty
    {"id": "example_007", "difficulty": 0.1},  # Low difficulty
    {"id": "example_008", "difficulty": 0.7},  # High difficulty
    {"id": "example_009", "difficulty": 0.05}, # Very low difficulty
]

print(f"✓ Sample dataset created with {len(sample_data)} examples")
print("Difficulty range:", f"{min(ex['difficulty'] for ex in sample_data):.2f} - {max(ex['difficulty'] for ex in sample_data):.2f}")

## Sample Dataset

Since the original script reads from external JSON files, we'll inline the sample data here to make the notebook self-contained. The dataset contains examples with varying difficulty levels that simulate different error probabilities.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

print("✓ DKWController class defined successfully!")

## DKW Controller Class

The `DKWController` class implements a decision-making system that uses the Dvoretzky-Kiefer-Wolfowitz inequality to maintain confidence bounds on error rates.

### Key Parameters:
- `epsilon_target`: Target error rate threshold (default: 0.10)
- `delta`: Confidence parameter for DKW bound (default: 0.05) 
- `min_samples`: Minimum observations before making decisions (default: 100)
- `hysteresis`: Buffer to prevent state oscillation (default: 0.05)

### Methods:
- `dkw_epsilon(n)`: Computes the DKW confidence bound for n samples
- `add_observation(error)`: Records a new error observation
- `decide()`: Makes fusion/fission decision based on current evidence

In [ ]:
"""Required imports for the DKW Controller implementation."""
import json
import numpy as np
from dataclasses import dataclass, field

print("✓ All imports loaded successfully!")

# DKW Controller Implementation

**Artifact ID:** experiment_001  
**Name:** method.py

This notebook demonstrates a DKW-guided fusion/fission controller implementation. The controller uses the Dvoretzky-Kiefer-Wolfowitz (DKW) inequality to provide statistical guarantees when deciding between fusion and fission states based on error observations.

## Overview

The DKW controller maintains confidence bounds on the empirical error rate and switches between:
- **Fusion state**: Aggressive/optimistic mode
- **Fission state**: Conservative/safe mode

The controller uses hysteresis to prevent oscillation between states.